# Работа с API YandexGPT

In [80]:
import requests
import json
import time

Для работы с API YandexGPT необходим идентификатор каталога и API-ключ для сервисного аккаунта Яндекс.Облака с ролью 'ai.languageModels.user'.

Подробнее о создании API-ключа: https://yandex.cloud/ru/docs/iam/concepts/authorization/api-key

In [81]:
yandex_cloud_catalog = ""
yandex_api_key = ""

Варианты модели:
- yandexgpt-lite
- yandexgpt
- summarization (для суммарищации текстов)

In [82]:
yandex_gpt_model = "yandexgpt-lite"

Структура промта:
- Общая инструкция (Ты ...)
- Запрос (Напиши ...)
- Контекст (Текст)
- Формат ответа

In [83]:
prompt = "Ты поэт и автор стихов для детей. Придумай стих про программиста из 4 строк. В ответе напиши только стих."

Параметры генерации
- temperature: чем выше значение этого параметра, тем более креативными и случайными будут ответы модели. Принимает значения от 0 до 1

In [84]:
temperature = 0.6

Для выполнения запроса необходимо сгенерировать body, указать url и заголовки

In [85]:
body = {
    "modelUri": f"gpt://{yandex_cloud_catalog}/{yandex_gpt_model}",
    "completionOptions": {
        "stream": False,
        "temperature": temperature,
        "maxTokens": "2000",
    },
    "messages": [{"role": "user", "text": prompt}],
}

url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Api-Key {yandex_api_key}",
}

Ответ возвращается в JSON-формате

In [86]:
response = requests.post(url, headers=headers, json=body)
response_json = json.loads(response.text)
print(response_json)

answer = response_json["result"]["alternatives"][0]["message"]["text"]
print(answer)

{'result': {'alternatives': [{'message': {'role': 'assistant', 'text': 'Программист всегда в работе,  \nЦелый день сидит в заботе.  \nЗнает он язык машин —  \nПишет код для всех вершин.'}, 'status': 'ALTERNATIVE_STATUS_FINAL'}], 'usage': {'inputTextTokens': '39', 'completionTokens': '33', 'totalTokens': '72'}, 'modelVersion': '18.01.2024'}}
Программист всегда в работе,  
Целый день сидит в заботе.  
Знает он язык машин —  
Пишет код для всех вершин.


### Добавления системного промпта

In [87]:
system_prompt = "Исправь ошибки в тексте. В ответе напиши только исправленный текст."
prompt = input()

In [88]:
body = {
    "modelUri": f"gpt://{yandex_cloud_catalog}/{yandex_gpt_model}",
    "completionOptions": {"stream": False, "temperature": 0, "maxTokens": "2000"},
    "messages": [
        {"role": "system", "text": system_prompt},
        {"role": "user", "text": prompt},
    ],
}

url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Api-Key {yandex_api_key}",
    "x-folder-id": yandex_cloud_catalog,
}

response = requests.post(url, headers=headers, json=body)
response_json = json.loads(response.text)
print(response_json)

answer = response_json["result"]["alternatives"][0]["message"]["text"]
print(answer)

{'result': {'alternatives': [{'message': {'role': 'assistant', 'text': 'Собака гуляет на поводке, а люди отдыхают.'}, 'status': 'ALTERNATIVE_STATUS_FINAL'}], 'usage': {'inputTextTokens': '40', 'completionTokens': '10', 'totalTokens': '50'}, 'modelVersion': '18.01.2024'}}
Собака гуляет на поводке, а люди отдыхают.


### Документация

https://yandex.cloud/ru/docs/foundation-models/operations/yandexgpt/create-prompt